In [1]:

import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt

In [2]:
TRAIN_TEST_SPLIT = 0.8

In [3]:
dataset_folder = '/usr4/cs640g/wyutong/ondemand/data/sys/dashboard/batch_connect/db/UTKFace/UTKFace'

In [4]:
dataset_dict = {'race_id':{0:'white',
                           1:'black',
                           2:'asian',
                           3:'indian',
                           4:'others'
                 },
                 'gender_id':{0:'male',
                              1:'female'
                     
                 }
               }

In [5]:
dataset_dict['gender_alias'] = dict((g, i) for i, g in dataset_dict['gender_id'].items())
dataset_dict['race_alias'] = dict((r, i) for i, r in dataset_dict['race_id'].items())

In [6]:
 def parse_info_from_file(path):
        try:
            filename = os.path.split(path)[1]
            filename = os.path.splitext(filename)[0]
            age, gender, race, _ = filename.split('_')
            return int(age), dataset_dict['gender_id'][int(gender)], dataset_dict['race_id'][int(race)]
        except Exception as ex:
            return None, None, None

In [7]:
def parse_dataset(dataset_path, ext='jpg'):     
    files = glob.glob(os.path.join(dataset_path, "*.%s" % ext))
    
    records = []
    for i,file in enumerate(files):
        info = parse_info_from_file(file)
        records.append(info)
        
    df = pd.DataFrame(records)
    df['file'] = files
    df.columns = ['age', 'gender', 'race', 'file']
    df = df.dropna()
    
    return df

In [8]:
df = parse_dataset(dataset_folder)
df.head()

,age,gender,race,file
0,47.0,female,indian,/usr4/cs640g/wyutong/ondemand/data/sys/dashboa...
1,1.0,female,white,/usr4/cs640g/wyutong/ondemand/data/sys/dashboa...
2,35.0,female,black,/usr4/cs640g/wyutong/ondemand/data/sys/dashboa...
3,26.0,female,asian,/usr4/cs640g/wyutong/ondemand/data/sys/dashboa...
4,29.0,male,white,/usr4/cs640g/wyutong/ondemand/data/sys/dashboa...


In [9]:
df = df[df.race != 'indian']

In [10]:
df.drop(columns=['gender'],inplace = True)
df.head()

,age,race,file
1,1.0,white,/usr4/cs640g/wyutong/ondemand/data/sys/dashboa...
2,35.0,black,/usr4/cs640g/wyutong/ondemand/data/sys/dashboa...
3,26.0,asian,/usr4/cs640g/wyutong/ondemand/data/sys/dashboa...
4,29.0,white,/usr4/cs640g/wyutong/ondemand/data/sys/dashboa...
5,52.0,white,/usr4/cs640g/wyutong/ondemand/data/sys/dashboa...


In [11]:

def plot_distribution(pd_series):
    labels = pd_series.value_counts().index.tolist()
    counts = pd_series.value_counts().values.tolist()
    print(labels,counts)

In [12]:
plot_distribution(df['race'])

['white', 'black', 'asian', 'others'] [3717, 1718, 1246, 660]


In [13]:
bins = [0, 10, 20, 30, 40, 60, 80, np.inf]
names = ['<10', '10-20', '20-30', '30-40', '40-60', '60-80', '80+']
age_binned = pd.cut(df['age'], bins, labels=names)
plot_distribution(age_binned)

['20-30', '30-40', '40-60', '<10', '60-80', '10-20', '80+'] [2333, 1342, 1324, 989, 643, 537, 173]


In [14]:
dataset_dict = {'race_id':{0:'white',
                           1:'black',
                           2:'asian',
                           3:'others'
                 }}
                 

In [15]:
#dataset_dict['gender_alias'] = dict((g, i) for i, g in dataset_dict['gender_id'].items())
dataset_dict['race_alias'] = dict((r, i) for i, r in dataset_dict['race_id'].items())

In [16]:
print(dataset_dict)

{'race_id': {0: 'white', 1: 'black', 2: 'asian', 3: 'others'}, 'race_alias': {'white': 0, 'black': 1, 'asian': 2, 'others': 3}}


In [17]:
df.head()

,age,race,file
1,1.0,white,/usr4/cs640g/wyutong/ondemand/data/sys/dashboa...
2,35.0,black,/usr4/cs640g/wyutong/ondemand/data/sys/dashboa...
3,26.0,asian,/usr4/cs640g/wyutong/ondemand/data/sys/dashboa...
4,29.0,white,/usr4/cs640g/wyutong/ondemand/data/sys/dashboa...
5,52.0,white,/usr4/cs640g/wyutong/ondemand/data/sys/dashboa...


In [19]:
from tensorflow.keras.utils import to_categorical
from PIL import Image

class UtkFaceDataGenerator():
    def __init__(self, df):
        self.df = df
        
    def generate_split_indexes(self):
        p = np.random.permutation(len(self.df))
        train_up_to = int(len(self.df) * TRAIN_TEST_SPLIT)
        train_idx = p[:train_up_to]
        test_idx = p[train_up_to:]        
        train_up_to = int(train_up_to * TRAIN_TEST_SPLIT)
        train_idx, valid_idx = train_idx[:train_up_to], train_idx[train_up_to:]
        
        #self.df['gender_id'] = self.df['gender'].map(lambda gender: dataset_dict['gender_alias'][gender])
        self.df['race_id'] = self.df['race'].map(lambda race: dataset_dict['race_alias'][race])        
        self.max_age = self.df['age'].max()
        
        return train_idx, valid_idx, test_idx
    
    def preprocess_image(self, img_path):
        im = Image.open(img_path)
        im = im.resize((IM_WIDTH, IM_HEIGHT))
        im = np.array(im) / 255.0       
        return im
        
    def generate_images(self, image_idx, is_training, batch_size=32):     
        images, ages, races = [], [], []
        while True:
            for idx in image_idx:
                person = self.df.iloc[idx]
                
                age = person['age']
                race = person['race_id']
                #gender = person['gender_id']
                file = person['file']
                
                im = self.preprocess_image(file)
                
                ages.append(age / self.max_age)
                races.append(to_categorical(race, len(dataset_dict['race_id'])))
                #genders.append(to_categorical(gender, len(dataset_dict['gender_id'])))
                images.append(im)
                
                if len(images) >= batch_size:
                    yield np.array(images), [np.array(ages), np.array(races)]
                    images, ages, races, genders = [], [], [], []
                    
            if not is_training:
                break

In [20]:
data_generator = UtkFaceDataGenerator(df)
train_idx, valid_idx, test_idx = data_generator.generate_split_indexes() 

In [21]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
import tensorflow as tf

class UtkMultiOutputModel():
    def make_default_hidden_layers(self, inputs):
        x = Conv2D(16, (3, 3), padding="same")(inputs)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)        
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)        
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)        
        return x  
    
    def build_race_branch(self, inputs, num_races):
        x = self.make_default_hidden_layers(inputs)        
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(num_races)(x)
        x = Activation("softmax", name="race_output")(x)        
        return x  
    
    
    def build_age_branch(self, inputs):            
        x = self.make_default_hidden_layers(inputs)        
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(1)(x)
        x = Activation("linear", name="age_output")(x)        
        return x 
    
    def assemble_full_model(self, width, height, num_races):       
        input_shape = (height, width, 3)        
        inputs = Input(shape=input_shape)        
        age_branch = self.build_age_branch(inputs)
        race_branch = self.build_race_branch(inputs, num_races)
        #gender_branch = self.build_gender_branch(inputs)        
        model = Model(inputs=inputs,
                     outputs = [age_branch, race_branch],
                     name="face_net")        
        return model

In [22]:
IM_WIDTH = IM_HEIGHT = 198

In [23]:
model = UtkMultiOutputModel().assemble_full_model(IM_WIDTH, IM_HEIGHT, num_races=len(dataset_dict['race_alias']))

In [29]:
init_lr = 1e-4
epochs = 100

In [25]:
from tensorflow.keras.optimizers import Adam

opt = Adam(lr=init_lr, decay=init_lr / epochs)
model.compile(optimizer=opt, 
              loss={
                  'age_output': 'mse', 
                  'race_output': 'categorical_crossentropy'
                  },
              
              loss_weights={
                  'age_output': 4., 
                  'race_output': 1.5
                  },
              
              metrics={
                  'age_output': 'mae', 
                  'race_output': 'accuracy'
                  })

/share/pkg.7/tensorflow/2.5.0/install/lib/SCC/../python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [42]:
from tensorflow import keras
model = keras.models.load_model("./Best_model_params.h5")

In [30]:
from tensorflow.keras.callbacks import ModelCheckpoint
#from livelossplot import PlotLossesKerasTF

batch_size = 32
valid_batch_size = 32

train_gen = data_generator.generate_images(train_idx, is_training=True, batch_size=batch_size)
valid_gen = data_generator.generate_images(valid_idx, is_training=True, batch_size=valid_batch_size)
checkpoint = ModelCheckpoint("Best_model_params.h5", monitor='val_loss')
callbacks = [checkpoint]

history = model.fit(train_gen,
                    steps_per_epoch=len(train_idx)//batch_size,
                    epochs=epochs,
                    callbacks=callbacks,
                    validation_data=valid_gen,
                    validation_steps=len(valid_idx)//valid_batch_size)
                    

Epoch 1/100
146/146 [==============================] - 25s 174ms/step - loss: 10.8614 - age_output_loss: 2.2072 - race_output_loss: 1.3550 - age_output_mae: 1.1589 - race_output_accuracy: 0.5435 - val_loss: 198.2869 - val_age_output_loss: 48.6513 - val_race_output_loss: 2.4545 - val_age_output_mae: 6.9393 - val_race_output_accuracy: 0.5026
Epoch 2/100
146/146 [==============================] - 15s 106ms/step - loss: 9.0900 - age_output_loss: 1.8432 - race_output_loss: 1.1449 - age_output_mae: 1.0503 - race_output_accuracy: 0.5978 - val_loss: 280.2472 - val_age_output_loss: 69.3705 - val_race_output_loss: 1.8434 - val_age_output_mae: 7.9350 - val_race_output_accuracy: 0.4505
Epoch 3/100
146/146 [==============================] - 15s 105ms/step - loss: 7.4829 - age_output_loss: 1.4796 - race_output_loss: 1.0429 - age_output_mae: 0.9432 - race_output_accuracy: 0.6366 - val_loss: 211.1392 - val_age_output_loss: 52.1715 - val_race_output_loss: 1.6354 - val_age_output_mae: 5.6821 - val_race_

146/146 [==============================] - 15s 105ms/step - loss: 1.4187 - age_output_loss: 0.1877 - race_output_loss: 0.4452 - age_output_mae: 0.3259 - race_output_accuracy: 0.8365 - val_loss: 76.6912 - val_age_output_loss: 18.8826 - val_race_output_loss: 0.7739 - val_age_output_mae: 1.9474 - val_race_output_accuracy: 0.7891
Epoch 26/100
146/146 [==============================] - 16s 107ms/step - loss: 1.2792 - age_output_loss: 0.1604 - race_output_loss: 0.4252 - age_output_mae: 0.3060 - race_output_accuracy: 0.8425 - val_loss: 75.2700 - val_age_output_loss: 18.5382 - val_race_output_loss: 0.7449 - val_age_output_mae: 1.9307 - val_race_output_accuracy: 0.7847
Epoch 27/100
146/146 [==============================] - 15s 104ms/step - loss: 1.2366 - age_output_loss: 0.1493 - race_output_loss: 0.4263 - age_output_mae: 0.2930 - race_output_accuracy: 0.8420 - val_loss: 61.6536 - val_age_output_loss: 15.1172 - val_race_output_loss: 0.7899 - val_age_output_mae: 1.6681 - val_race_output_accurac

146/146 [==============================] - 16s 107ms/step - loss: 0.4665 - age_output_loss: 0.0201 - race_output_loss: 0.2573 - age_output_mae: 0.1092 - race_output_accuracy: 0.9077 - val_loss: 16.0850 - val_age_output_loss: 3.7534 - val_race_output_loss: 0.7142 - val_age_output_mae: 0.6562 - val_race_output_accuracy: 0.7977
Epoch 50/100
146/146 [==============================] - 15s 106ms/step - loss: 0.4433 - age_output_loss: 0.0193 - race_output_loss: 0.2441 - age_output_mae: 0.1073 - race_output_accuracy: 0.9144 - val_loss: 19.0166 - val_age_output_loss: 4.5060 - val_race_output_loss: 0.6618 - val_age_output_mae: 0.7413 - val_race_output_accuracy: 0.8177
Epoch 51/100
146/146 [==============================] - 16s 111ms/step - loss: 0.4348 - age_output_loss: 0.0185 - race_output_loss: 0.2404 - age_output_mae: 0.1044 - race_output_accuracy: 0.9182 - val_loss: 17.9597 - val_age_output_loss: 4.2292 - val_race_output_loss: 0.6954 - val_age_output_mae: 0.7134 - val_race_output_accuracy: 

146/146 [==============================] - 16s 109ms/step - loss: 0.2830 - age_output_loss: 0.0132 - race_output_loss: 0.1535 - age_output_mae: 0.0882 - race_output_accuracy: 0.9458 - val_loss: 5.8796 - val_age_output_loss: 1.1920 - val_race_output_loss: 0.7410 - val_age_output_mae: 0.3359 - val_race_output_accuracy: 0.8047
Epoch 74/100
146/146 [==============================] - 15s 106ms/step - loss: 0.2794 - age_output_loss: 0.0129 - race_output_loss: 0.1519 - age_output_mae: 0.0865 - race_output_accuracy: 0.9516 - val_loss: 5.9754 - val_age_output_loss: 1.2156 - val_race_output_loss: 0.7419 - val_age_output_mae: 0.3526 - val_race_output_accuracy: 0.8047
Epoch 75/100
146/146 [==============================] - 15s 106ms/step - loss: 0.2726 - age_output_loss: 0.0131 - race_output_loss: 0.1467 - age_output_mae: 0.0876 - race_output_accuracy: 0.9493 - val_loss: 3.8971 - val_age_output_loss: 0.7054 - val_race_output_loss: 0.7171 - val_age_output_mae: 0.2645 - val_race_output_accuracy: 0.8

146/146 [==============================] - 15s 104ms/step - loss: 0.1984 - age_output_loss: 0.0099 - race_output_loss: 0.1058 - age_output_mae: 0.0766 - race_output_accuracy: 0.9668 - val_loss: 4.2711 - val_age_output_loss: 0.7494 - val_race_output_loss: 0.8489 - val_age_output_mae: 0.2382 - val_race_output_accuracy: 0.8030
Epoch 98/100
146/146 [==============================] - 16s 107ms/step - loss: 0.1900 - age_output_loss: 0.0097 - race_output_loss: 0.1009 - age_output_mae: 0.0753 - race_output_accuracy: 0.9700 - val_loss: 3.0754 - val_age_output_loss: 0.4623 - val_race_output_loss: 0.8174 - val_age_output_mae: 0.1997 - val_race_output_accuracy: 0.8082
Epoch 99/100
146/146 [==============================] - 16s 107ms/step - loss: 0.1809 - age_output_loss: 0.0100 - race_output_loss: 0.0938 - age_output_mae: 0.0768 - race_output_accuracy: 0.9711 - val_loss: 2.3703 - val_age_output_loss: 0.2895 - val_race_output_loss: 0.8082 - val_age_output_mae: 0.1731 - val_race_output_accuracy: 0.8

In [33]:
test_batch_size = 128
test_generator = data_generator.generate_images(test_idx, is_training=False, batch_size=test_batch_size)
age_pred, race_pred = model.predict(test_generator, 
                                                           steps=len(test_idx)//test_batch_size)

In [35]:
test_generator = data_generator.generate_images(test_idx, is_training=False, batch_size=test_batch_size)
samples = 0
images, age_true, race_true = [], [], []
for test_batch in test_generator:
    image = test_batch[0]
    labels = test_batch[1]
    
    images.extend(image)
    age_true.extend(labels[0])
    race_true.extend(labels[1])
    
age_true = np.array(age_true)
race_true = np.array(race_true)


race_true = race_true.argmax(axis=-1)
race_pred= race_pred.argmax(axis=-1)

age_true = age_true * data_generator.max_age
age_pred = age_pred * data_generator.max_age

In [36]:
from sklearn.metrics import classification_report
cr_race = classification_report(race_true, race_pred, target_names=dataset_dict['race_alias'].keys())
print(cr_race)

              precision    recall  f1-score   support

       white       0.81      0.93      0.87       717
       black       0.92      0.84      0.87       330
       asian       0.80      0.79      0.80       232
      others       0.48      0.22      0.30       129

    accuracy                           0.82      1408
   macro avg       0.75      0.69      0.71      1408
weighted avg       0.81      0.82      0.81      1408



In [41]:
from sklearn.metrics import r2_score
print('R2 score for age: ', r2_score(age_true, age_pred))

R2 score for age:  -16.572098745946555
